In [23]:
from Bio import SeqIO
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os

In [24]:
### GLOBAL VARIABLES

#LENGTH SELECTION
min_length = 195
max_length = 205

#PRIMER
forward_primer = 'TTGCTTGGGTTTTTCAGCGT'
reverse_primer = 'GCCGCAAGTTTAACTCCAACA'

#PATH
df = pd.read_csv('//home/qichaotan/CODE_ARCHIVE/code_archive/haplotype_network_construction/COX3_ASV_table.csv') #Input OTU table, csv format, column: Sequence, row: Sample name
file_list = os.listdir('/home/qichaotan/COX3_dada2') # Original sample names, convert sample names to indexes, in order to shorten the name for phylip files. 


In [25]:
#Mapper from sample name to index
site2index = dict()
for i in file_list:
    location_info = i.split('_')
    site2index.update({location_info[0]:location_info[1]})  

In [26]:
ASV_list = df.columns.to_list()[1:]

#Clean out sample based on length
ASV_list = [seq for seq in ASV_list if len(seq) >= min_length and len(seq) <= max_length and seq.startswith(forward_primer) == True and seq.endswith(reverse_primer) == True]

ASV_alias = []
for i in range(len(ASV_list)):
    ASV_alias.append(str('ASV'+str(i+1)))
ASV_mapper = dict(zip(ASV_list, ASV_alias))
ASV_reverse_mapper = dict(zip(ASV_alias, ASV_list))
df.rename(columns = {'Unnamed: 0':'Site'}, inplace=True)
ASV_list.insert(0,'Site')
df = df[ASV_list]
df.rename(columns = ASV_mapper, inplace = True)

In [27]:
with open('COX3_ASV_soft.fasta', 'w') as output_handle:
    for i in df['Site'].to_list()[1:]:
        temp_df = df[df['Site'] == i]
        for j in ASV_alias:
            if temp_df[j].values != 0:
                record = SeqRecord(
                    Seq(str(ASV_reverse_mapper[j])),
                    id = str(j) + '_' + str(site2index[i]),
                    description = "Emiliania huxleyi COX3 gene" + ' ' + str(j) 
                )
                SeqIO.write(record, output_handle, 'fasta')

In [28]:
with open('COX3_ASV_hard.fasta', 'w') as output_handle:
    for key, value in ASV_reverse_mapper.items():
        record = SeqRecord(
            Seq(str(value)),
            id = key,
            description = "Emiliania huxleyi COX3 gene" + ' ' + str(key)
        )
        SeqIO.write(record, output_handle, 'fasta')

In [29]:
with open('Cleaned_ASV_table.csv','w') as output_handle:
    df.to_csv(output_handle)